> This notebook accompanies the Scailable tutorial [Creating ONNX pipelines from scratch](). Please see the tutorial for details.

The aim of this notebook is to create an ONNX pipeline which, given a vector containing the yard-size, indoor-area, and number of rooms of a house (`(yard,area,rooms)`):

* Predicts the price of the house (using a trained model presented in "Block 0" below),
* Checks whether the predicted price is smaller than 400.000, and the yard-size is larger than 0, 
* Returns a Boolean indicating whether or not the house if of interest.


# Block 0: Preliminaries
The code block below provides our startingpoint by fitting a simple linear regression model mapping the standardized feature vector `(yard, area, rooms)` to predict the log of the observed outcome `(price)`.

In [100]:
import numpy as np
from sklearn import linear_model as lm

# Open the training data:
data = np.loadtxt(open("houses.csv", "rb"), delimiter=",", skiprows=1)

# Retreive feature vectors and outcomes:
datX = data[:, [1,2,3]]  # Input features (yard, area, rooms)
datY = data[:, [0]]  # Price

# Standardize the inputs:
barX = np.mean(datX, 0)  # Mean for each of the inputs
sdX = np.std(datX, 0)  # Sd for each of the inputs
datZ = (datX - barX) / sdX

# Log transform the output
logY = np.log(datY)

# Fit a linear model
lin_mod = lm.LinearRegression()
lin_mod.fit(datZ, logY)

# retrieve intercept and fitted coefficients:
intercept = lin_mod.intercept_
beta = lin_mod.coef_

print("--- Values retrieved from training --- ")
print("For input statdardization / pre-processing we need:")
print(" - The column means {}".format(barX))
print(" - The column sds {}".format(sdX))

print("For the prediction we need:")
print(" - The estimated coefficients: {}".format(beta))
print(" - The intercept: {}".format(intercept))

# store the training results in an object to make the code more readable later on:
training_results = {
    "barX" : barX.astype(np.float32),
    "sdX" : sdX.astype(np.float32),
    "beta" : beta.astype(np.float32),
    "intercept" : intercept.astype(np.float32),
}

# And, also creating the constraints (for usage in block 3):
constraints = {
    "maxprice" : np.array([400000]),
    "minyard" : np.array([1]),
}

--- Values retrieved from training --- 
For input statdardization / pre-processing we need:
 - The column means [238.08108108 124.13513514   4.71744472]
 - The column sds [556.90768344  55.94648126   1.8576531 ]
For the prediction we need:
 - The estimated coefficients: [[ 0.03990859  0.36480886 -0.00673619]]
 - The intercept: [12.69092987]


The following code is not neccesary for our main aim, but illustrates how to construct a prediction and evaluate our decision rules given a single obeservation (i.e., the properties of a single house):

In [101]:
# Get the data from a single house
first_row_example = data[1,:]
input_example = first_row_example[[1,2,3]]  # The features
output_example = first_row_example[0]  # The observed price  

# 1. Standardize input for input to the model:
standardized_input_example = (input_example - training_results['barX'])/ training_results['sdX']

# 2. Predict the *log* price (using a dot product and the intercept)
predicted_log_price_example = training_results['intercept'] + np.dot(standardized_input_example, training_results['beta'].flatten())


# Compute the actual prediction on the original scale
predicted_price_example = np.exp(predicted_log_price_example)
print("Observed price: {}, predicted price: {}".format(output_example, predicted_price_example))

# See if it is interesting according to our simple decision rules:
interesting = input_example[1] > 0 and predicted_price_example < 400000
print("Interesting? {}".format(interesting))


Observed price: 500000.0, predicted price: [422147.75]
Interesting? [False]


# Block 1: Constructing the pre-processing pipeline
This block of code constructs the pre-processing pipeline for our scenario in ONNX. Refer to the [Creating ONNX pipelines from scratch]() tutorial for naming conventions.

In [106]:
# All the neccesary imports for blocks 1-X
from onnx import helper as h
from onnx import TensorProto as tp
from onnx import checker
from onnx import save

In [107]:
# The required constants:
c1 = h.make_node('Constant', inputs=[], outputs=['c1'], name="c1-node", 
        value=h.make_tensor(name="c1v", data_type=tp.FLOAT, 
        dims=training_results['barX'].shape, 
        vals=training_results['barX'].flatten()))

c2 = h.make_node('Constant', inputs=[], outputs=['c2'], name="c2-node", 
        value=h.make_tensor(name="c2v", data_type=tp.FLOAT, 
        dims=training_results['sdX'].shape, 
        vals=training_results['sdX'].flatten()))

# The functional nodes:
n1 = h.make_node('Sub', inputs=['x', 'c1'], outputs=['xmin'], name='n1')
n2 = h.make_node('Div', inputs=['xmin', 'c2'], outputs=['zx'], name="n2")

# Create the graph
g1 = h.make_graph([c1, n1, c2, n2], 'preprocessing',
        [h.make_tensor_value_info('x', tp.FLOAT, [3])],
        [h.make_tensor_value_info('zx', tp.FLOAT, [3])])

# Create the model and check
m1 = helper.make_model(g1, producer_name='scailable-demo')
checker.check_model(m1)

# Save the model
save(m1, 'pre-processing.onnx')

In [108]:
# A few lines to evaluate the stored model, useful for debugging:
import onnxruntime as rt

# test
sess = rt.InferenceSession("pre-processing.onnx")  # Start the inference session and open the model
xin = input_example.astype(np.float32)  # Use the input_example from block 0 as input
zx = sess.run(["zx"], {"x": xin})  # Compute the standardized output

print("Check:")
print("The standardized input using onnx pipeline is: {}".format(zx))
print(" - Compare to standardized first row in block 0: {}".format(datZ[1,:]))

Check:
The standardized input using onnx pipeline is: [array([-0.04503635,  0.76617634,  2.3053577 ], dtype=float32)]
 - Compare to standardized first row in block 0: [-0.04503634  0.76617624  2.30535792]


# Block 2: Constructing the inference model
The code block below constructs the ONNX inference task which, from the standardized input `zx` predicts the price on a log scale.

In [94]:
# The constants:
c3 = h.make_node('Constant', inputs=[], outputs=['c3'], name="c3-node", 
        value=h.make_tensor(name="c3v", data_type=tp.FLOAT, 
        dims=training_results['beta'].shape, 
        vals=training_results['beta'].flatten()))

c4 = h.make_node('Constant', inputs=[], outputs=['c4'], name="c4-node", 
        value=h.make_tensor(name="c4v", data_type=tp.FLOAT, 
        dims=training_results['intercept'].shape, 
        vals=training_results['intercept'].flatten()))

# The operating nodes, Multiply, reduceSum, and Add
n3 = h.make_node('Mul', inputs=['zx', 'c3'], outputs=['mulx'], name="multiplyBeta")
n4 = h.make_node('ReduceSum', inputs=['mulx'], outputs=['sumx'], name="reduceSum", keepdims=0)
n5 = h.make_node('Add', inputs=['sumx', 'c4'], outputs=['yhatlog'], name='addIntercept')

# The graph
g2 = h.make_graph([c3, c4, n3, n4, n5], 'linear_regression',
       [h.make_tensor_value_info('zx', tp.FLOAT, [3])],
       [h.make_tensor_value_info('yhatlog', tp.FLOAT, [1])])

# The model and check:
m2 = h.make_model(g2, producer_name='scailable-demo')
checker.check_model(m2)

# Save the model
save(m2, 'linear-regression.onnx')

In [95]:
# test
sess = rt.InferenceSession("linear-regression.onnx")  # Start the inference session and open the model
xin = standardized_input_example.astype(np.float32)  # Use the input_example from block 0 as input
yhatlog = sess.run(["yhatlog"], {"zx": xin})  # Compute the standardized output

print("Check:")
print("The log predicted price from ONNX is: {}".format(yhatlog))
print(" - Compare to analysis in block 0: {}".format(predicted_log_price_example))

Check:
The log predicted price from ONNX is: [array([12.953111], dtype=float32)]
 - Compare to analysis in block 0: [12.953111]


# Block 3: Constructing the post-processing pipeline
Going from the log predicted price of the house to the actual verdict.

In [96]:
# Constants (note using the constraints object created in block 0 above)
c5 = h.make_node('Constant', inputs=[], outputs=['c5'], name="c5-node", 
        value=h.make_tensor(name="c5v", data_type=tp.FLOAT, 
        dims=constraints['maxprice'].shape, 
        vals=constraints['maxprice'].flatten()))
c6 = h.make_node('Constant', inputs=[], outputs=['c6'], name="c6-node", 
        value=h.make_tensor(name="c6v", data_type=tp.FLOAT, 
        dims=constraints['minyard'].shape, 
        vals=constraints['minyard'].flatten()))

# Auxilary constants for the slice operator:
caux1 = h.make_node('Constant', inputs=[], outputs=['caux1'], name="caux1-node",
        value=h.make_tensor(name='caux1v', data_type=tp.INT32,
        dims=np.array([0]).shape, vals=np.array([0]).flatten()))
caux2 = h.make_node('Constant', inputs=[], outputs=['caux2'], name="caux2-node",
        value=h.make_tensor(name='caux2v', data_type=tp.INT32,
        dims=np.array([1]).shape, vals=np.array([1]).flatten()))
caux3 = h.make_node('Constant', inputs=[], outputs=['caux3'], name="caux3-node",
        value=h.make_tensor(name='caux3v', data_type=tp.INT32,
        dims=np.array([0]).shape, vals=np.array([0]).flatten()))
caux4 = h.make_node('Constant', inputs=[], outputs=['caux4'], name="caux4-node",
        value=h.make_tensor(name='caux4v', data_type=tp.INT32,
        dims=np.array([1]).shape, vals=np.array([1]).flatten()))
            
# Nodes:
n6 = h.make_node('Exp', inputs=['yhatlog'], outputs=['yhat'], name='exponent')
n7 = h.make_node('Less', inputs=['yhat', 'c5'], outputs=['price_ok'], name='priceLess')

n8 = h.make_node('Slice', inputs=['x', 'caux1', 'caux2', 'caux3', 'caux4'], outputs=['yard'],)
n9 = h.make_node('Less', inputs=['c6', 'yard'], outputs=['yard_ok'], name="yardMore") # note reversal

n10 = h.make_node('And', inputs=['price_ok', 'yard_ok'], outputs=['result'], name='andBools')

# The graph
g3 = h.make_graph([c5, c6, caux1, caux2, caux3, caux4, n6, n7, n8, n9, n10], 'postprocessing',
       [h.make_tensor_value_info('x', tp.FLOAT, [3]), h.make_tensor_value_info('yhatlog', tp.FLOAT, [1])],
       [h.make_tensor_value_info('result', tp.BOOL, [1])])

# The model and check:
m3 = h.make_model(g3, producer_name='scailable-demo')
checker.check_model(m3)

# Save the model
save(m3, 'post-processing.onnx')

In [97]:
# test
sess = rt.InferenceSession("post-processing.onnx")  # Start the inference session and open the model
x = input_example.astype(np.float32)  # Use the input_example from block 0 as input

yhatlog = np.array(yhatlog).flatten()
result = sess.run(["result"], {"x": x, "yhatlog" : yhatlog})  # Compute the standardized output

print("Check:")
print("Predicted price {} and yardsize {} are appealing {}.".format(np.exp(yhatlog), input_example[0], result))

Check:
Predicted price [422147.75] and yardsize 213.0 are appealing [array([False])].


# Block 4: Creating a single graph for all operations
While we could exectue each of the processing blocks defined above in turn, we can also create a single model containing the full inference pipeline...

In [98]:
g_full = h.make_graph([c1, n1, c2, n2, c3, c4, n3, n4, n5, c5, c6, caux1, caux2, caux3, caux4, n6, n7, n8, n9, n10], 
        'fullpipeline',
        [h.make_tensor_value_info('x', tp.FLOAT, [3])],
        [h.make_tensor_value_info('result', tp.BOOL, [1])])

m_full = h.make_model(g_full, producer_name='scailable-demo')
checker.check_model(m_full)

# Save the model
save(m_full, 'full-pipeline.onnx')

In [99]:
# test
sess = rt.InferenceSession("full-pipeline.onnx")  # Start the inference session and open the model
xin = input_example.astype(np.float32)  # Use the input_example from block 0 as input

yhatlog = np.array(yhatlog).flatten()
result = sess.run(["result"], {"x": xin})  # Compute the standardized output

print("Check:")
print("Example {} is appealing: {}.".format(xin, result))

Check:
Example [213. 167.   9.] is appealing: [array([False])].
